In [3]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from sqlalchemy import create_engine
import pyodbc

In [4]:
print(pyodbc.drivers())

['SQL Server', 'MySQL ODBC 8.0 ANSI Driver', 'MySQL ODBC 8.0 Unicode Driver', 'SQL Server Native Client 11.0', 'SQL Server Native Client RDA 11.0', 'Oracle in OraDB21Home1', 'ODBC Driver 18 for SQL Server', 'ODBC Driver 17 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)']


In [60]:
driver = '{ODBC Driver 17 for SQL Server}'
host = 'HOAIBAO'
database = 'Wisdom_rubberProject_Practice_3'
trusted_connection = 'yes'
# Create the connection string
connection_string = f"DRIVER={driver};SERVER={host};DATABASE={database};Trusted_Connection={trusted_connection}"


In [9]:
# connection = pyodbc.connect(connection_string)
# connection.close()

In [64]:
def extract_from_mssql(connection_str):
    connection = pyodbc.connect(connection_str)
    print('connected')
    data_dict = {}
    # input the connection string
    # retrive tables' name
    sql_retrive_table_names = "select name from sys.tables where name <> 'sysdiagrams';"
    df_tableNames = pd.read_sql(sql_retrive_table_names, connection)
    # print(df_tableNames)
    for table in df_tableNames['name']:
        sql_retrive_data = f'select * from [{table}]'
        data_dict[table] = pd.read_sql(sql_retrive_data, connection)
        
    # create a dictionary with keys are name of the tables
    # retrive data, put into the dictionary as dataframe
    # return a dictionary with keys are name of the tables and values are content inside each table
    connection.close()
    print('disconnected')
    return data_dict

In [65]:
data_raw = extract_from_mssql(connection_string)

connected
disconnected


C:\Users\tranh\AppData\Local\Temp\ipykernel_18696\1180596050.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tableNames = pd.read_sql(sql_retrive_table_names, connection)
C:\Users\tranh\AppData\Local\Temp\ipykernel_18696\1180596050.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_dict[table] = pd.read_sql(sql_retrive_data, connection)
C:\Users\tranh\AppData\Local\Temp\ipykernel_18696\1180596050.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_dict[table] = pd.read_sql(sql_retrive_data, connection)
C

In [72]:
def transform(data_dict):
    # input: a dict of tables name and data
    for key, df in data_dict.items():
        print(
            f'table: {key}\n',
            df.info()
        )
    # input: a table of data?
    return # clean data

In [73]:
transform(data_raw)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   EnvironmentID   5 non-null      int64         
 1   RobotTappingID  5 non-null      int64         
 2   WindDirection   5 non-null      object        
 3   WindSpeed       5 non-null      float64       
 4   Temperature     5 non-null      float64       
 5   Rainfall        5 non-null      float64       
 6   Time            5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 408.0+ bytes
table: Environment
 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   DroneID  5 non-null      int64
 1   RobotID  5 non-null      int64
 2   ScsID    5 non-null      int64
dtypes: int64(3)
memory usage: 248.0 byte

In [71]:
def load(dw_conn_str, data_dict): # push data to data warehouse
    # input: a connection string to data warehouse
    # retrive tablename from the data warehouse
    connection = pyodbc.connect(connection_string)
    #
    
    
    connection.close()
    return # clean data